In [ ]:
!pip install -qU langchain langchain-community langchain-classic
!pip install -qU transformers accelerate bitsandbytes sentencepiece gradio faiss-cpu
!pip install -qU torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [ ]:
import os
import time
import math
import re
import torch
import gradio as gr
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

# ======================================================
# 🔴 STABLE IMPORTS (No Agent Classes)
# ======================================================
from langchain_community.llms import HuggingFacePipeline
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import ConversationSummaryMemory
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# ======================================================
# 1️⃣ Load LLM (FIXED Quantization)
# ======================================================
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

print("🔄 Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("🔄 Configuring 4-bit Quantization...")
# FIX: Use explicit configuration to solve the version mismatch error
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

print("🔄 Loading Model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config, # FIX: Pass config object instead of True
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

model.eval()
print("✅ LLM loaded")

generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

# ======================================================
# 2️⃣ Embeddings & Helpers
# ======================================================
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

BASE_DIR = "memory_store"
os.makedirs(BASE_DIR, exist_ok=True)

def load_store(path):
    if os.path.exists(path):
        return FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)
    return FAISS.from_texts(["Initialize"], embeddings)

def save_store(store, path):
    store.save_local(path)

# ======================================================
# 4️⃣ Importance Scoring
# ======================================================
def score_importance(text):
    prompt = f"""
Score the importance of this interaction (0-10).
Respond ONLY with a number.

{text}
"""
    result_list = generation_pipeline(prompt)
    result = result_list[0]['generated_text'].strip()
    match = re.search(r'\d+', result)
    if match:
        return min(10, max(0, int(match.group())))
    return 0

# ======================================================
# 5️⃣ Memory Decay Logic
# ======================================================
DECAY_LAMBDA = 0.15

def decay_score(importance, timestamp):
    age_days = (time.time() - timestamp) / 86400
    return importance * math.exp(-DECAY_LAMBDA * age_days)

# ======================================================
# 🔧 CUSTOM AGENT TOOLS
# ======================================================

def calculator_tool(expr: str) -> str:
    try:
        return str(eval(expr))
    except:
        return "Invalid expression"

# ======================================================
# 🧠 CUSTOM ROUTER LOGIC
# ======================================================
def decide_tool(message: str) -> str:
    prompt = f"""
You are a router. Choose the best tool for the user message.

Tools available:
1. calculator - If user asks for math calculation.
2. memory - If user asks "What do you remember?" or "My preferences".
3. chat - For general conversation.

User message: "{message}"

Answer ONLY with one word: calculator, memory, or chat.
"""
    res = generation_pipeline(prompt)[0]['generated_text'].strip().lower()

    if "calc" in res: return "calculator"
    if "mem" in res: return "memory"
    return "chat"

# ======================================================
# 7️⃣ User Session Builder
# ======================================================
def get_session(user_id):
    user_dir = os.path.join(BASE_DIR, user_id)
    os.makedirs(user_dir, exist_ok=True)

    facts_path = os.path.join(user_dir, "facts")
    store = load_store(facts_path)

    memory = ConversationSummaryMemory(llm=llm)
    conversation = ConversationChain(llm=llm, memory=memory)

    return conversation, store, facts_path

# ======================================================
# 8️⃣ Chat Logic
# ======================================================
def chat(message, history, user_id):
    conversation, store, store_path = get_session(user_id)

    # 1. Decide which tool/route to take
    route = decide_tool(message)
    print(f"🤖 Router Decision: {route}")

    # 2. Execute based on route
    if route == "calculator":
        try:
            extract_prompt = f"Extract only the math expression from this: {message}"
            expr = generation_pipeline(extract_prompt)[0]['generated_text'].strip()

            result = calculator_tool(expr)
            response = f"The answer is {result}."
        except Exception as e:
            response = "I couldn't do the math. Please provide the expression clearly."

    elif route == "memory":
        docs = store.similarity_search("", k=3)
        retrieved = "\n".join([d.page_content for d in docs])
        if not retrieved:
            response = "I don't have any stored memories for you yet."
        else:
            summary_prompt = f"Summarize these memories for the user:\n{retrieved}"
            response = generation_pipeline(summary_prompt)[0]['generated_text'].strip()

    else:
        # 3. Standard Chat with Memory Context
        docs = store.similarity_search(message, k=5)
        scored_docs = []

        for d in docs:
            meta = d.metadata
            importance = meta.get("importance", 0)
            timestamp = meta.get("timestamp", time.time())
            score = decay_score(importance, timestamp)
            scored_docs.append((score, d.page_content))

        scored_docs.sort(reverse=True)
        context = "\n".join([c for _, c in scored_docs[:3]])

        augmented_input = f"""
Relevant memories (context):
{context}

User message:
{message}
"""

        response = conversation.predict(input=augmented_input)

    # Clean up response
    for stop_string in ["\nUser:", "\nHuman:", "\nAI:"]:
        if stop_string in response:
            response = response.split(stop_string)[0].strip()

    interaction = f"User: {message}\nAssistant: {response}"

    if route != "calculator":
        importance = score_importance(interaction)
        if importance >= 6:
            store.add_texts(
                [interaction],
                metadatas=[{
                    "importance": importance,
                    "timestamp": time.time()
                }]
            )
            save_store(store, store_path)
            print(f"💾 Saved memory (Score: {importance}) for {user_id}")

    return response

# ======================================================
# 9️⃣ Gradio UI
# ======================================================
demo = gr.ChatInterface(
    fn=chat,
    additional_inputs=[gr.Textbox(label="User ID", value="user_1")],
    title="🧠 Custom Agent with Decaying Memory + Tools",
    description="Manual Router Agent + FAISS + importance decay (Stable Version)",
)

demo.launch(share=True)

🔄 Loading Tokenizer...


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6a7c0059823c2cc643.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
